In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import geopandas as gpd
import pandas as pd

In [2]:
# input Hamilton county geo file
ham_cbg = gpd.read_file('../data/Tennessee Census Block Groups/tl_2020_47_bg.shp')
ham_cbg = ham_cbg[ham_cbg.COUNTYFP=='065']
ham_cbg = ham_cbg.to_crs('epsg:4326')

In [14]:
#finding all buildings
tags = {'building': True}
buildings = ox.geometries_from_bbox(34.854382885097905, 35.935532323321, -84.19759521484375, -85.553161621093756, tags)

In [17]:
#aggregating all residential tags
pd.set_option('display.max_columns', None)
res_build = buildings[(buildings.building == 'residential') | (buildings.building == 'bungalow') | (buildings.building == 'cabin') | (buildings.building == 'dormitory') | (buildings.building == 'hotel') | (buildings.building == 'house') | (buildings.building == 'semidetached_house') | (buildings.building == 'barracks') | (buildings.building == 'farm') | (buildings.building == 'ger') | (buildings.building == 'houseboat') | (buildings.building == 'static_caravan') | (buildings.building == 'terrace')].reset_index()[['osmid', 'geometry', 'nodes', 'building', 'name', 'source']].sjoin(ham_cbg)
res_build.geometry = res_build.geometry.apply(lambda x: x.centroid if x.geom_type =='Polygon' else x)

In [7]:
#saving residential buildings
res_build.to_csv('ham_residential_buildings.csv', index=False)

In [87]:
pd.set_option('display.max_columns', None)

#work tags

com_build = buildings[(buildings.building == 'commercial') | (buildings.building == 'industrial') | (buildings.building == 'kiosk') | (buildings.building == 'office') | (buildings.building == 'retail') | (buildings.building == 'supermarket') | (buildings.building == 'warehouse')].reset_index()[['osmid', 'geometry', 'nodes', 'building', 'name', 'source']].sjoin(ham_cbg)
civ_build = buildings[(buildings.building == 'bakehouse') | (buildings.building == 'civic') | (buildings.building == 'college') | (buildings.building == 'fire_station') | (buildings.building == 'government') | (buildings.building == 'hospital') | (buildings.building == 'kindergarten') | (buildings.building == 'public') | (buildings.building == 'school') | (buildings.building == 'train_station') | (buildings.building == 'transportation') | (buildings.building == 'university')].reset_index()[['osmid', 'geometry', 'nodes', 'building', 'name', 'source']].sjoin(ham_cbg)

com_build.geometry = com_build.geometry.apply(lambda x: x.centroid if x.geom_type =='Polygon' else x)
civ_build.geometry = civ_build.geometry.apply(lambda x: x.centroid if x.geom_type =='Polygon' else x)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [60]:
def func(row):
    str(Point(gpd.points_from_xy(row.INTPTLAT, row.INTPTLON)[0]))

In [66]:
#converting the default internal point of each cbg to a shapely Point
res_build['intpt'] = res_build[['INTPTLAT', 'INTPTLON']].apply(lambda x: func, axis=1)

## adding safegraph poi locations

In [39]:
# add safegraph extracted data
sg = gpd.read_file('path to safegraph file') # path removed due to privacy concerns
sg.longitude = sg.longitude.astype(float)
sg.latitude = sg.latitude.astype(float)
geom = [Point(xy) for xy in zip(sg.longitude, sg.latitude)]
sg = gpd.GeoDataFrame(sg, geometry = geom, crs='epsg:4326')

In [123]:
#adding in safegraph POI locations to OSM work locations

t1 = (sg[['poi_cbg', 'geometry']].rename({'poi_cbg':'GEOID'},axis=1).append(com_build[['GEOID', 'geometry']]).append(civ_build[['GEOID', 'geometry']]))
t1.GEOID = t1.GEOID.astype(str).apply(lambda x: x.split('.')[0])

# saving work buildings to file
t1.to_csv('work_loc_poi_com_civ.csv', index=False)